In [1]:
from run_parallel import process_parallel_bold
import pandas as pd
import logging

# Silence pdfminer and pdfplumber logs
logging.getLogger("pdfminer").setLevel(logging.ERROR)
logging.getLogger("pdfplumber").setLevel(logging.ERROR)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:

df_all_granted, dic_questions = process_parallel_bold('data/granted/')
df_all_granted['final_visa_status'] = 'granted'

df_all_refused, dic_questions = process_parallel_bold('data/refused/',
                                field_order=list(dic_questions.keys()),
                                field_labels=dic_questions)
df_all_refused['final_visa_status'] = 'refused'

Processing: 100%|██████████| 4/4 [00:34<00:00,  8.64s/it]
/Users/albafranco/Documents/Python/visa-scoring/run_parallel.py:284: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'': np.nan}, inplace=True)
Processing: 100%|██████████| 4/4 [00:50<00:00, 12.66s/it]
/Users/albafranco/Documents/Python/visa-scoring/run_parallel.py:284: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'': np.nan}, inplace=True)
/Users/albafranco/Documents/Python/visa-scoring/run_parallel.py:287: FutureWarning: Downcasting behavior in `rep

In [20]:
print('df_all_granted',df_all_granted.shape)
print('df_all_refused',df_all_refused.shape)

df_all = pd.concat([df_all_granted,df_all_refused]).reset_index(drop=True)

print('df_all',df_all.shape)

df_all_granted (4, 152)
df_all_refused (4, 210)
df_all (8, 228)


In [21]:
def postprocess_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Post-procesa las celdas que contienen ';' de la siguiente manera:
      - Separa la cadena por ';'
      - Si el último segmento es un entero <100 y el penúltimo es 'si', 'no' o 'yes',
        devuelve el penúltimo.
      - En cualquier otro caso (incluyendo cuando no se cumplan esas condiciones),
        devuelve el valor original.
    """
    def _clean_cell(v):
        if isinstance(v, str) and ';' in v:
            parts = [p.strip() for p in v.split(';') if p.strip()]
            if len(parts) >= 2:
                last = parts[-1]
                try:
                    num = int(last)
                    penult = parts[-2].lower()
                    if num < 100 and penult in ('si', 'no', 'yes'):
                        return parts[-2]  # devolvemos penúltimo con su mayúsc/minúsc original
                    else:
                        if last in ('si', 'no', 'yes'):
                            return last
                        else:
                            return v
                except ValueError:
                    # si el último no es número, devolvemos el valor original completo
                    return v
        return v

    return df.applymap(_clean_cell)

def unify_suffix_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Para cada par de columnas base / base_<suffix>:
      - Rellena los NaN de 'base_<suffix>' con los valores de 'base'
      - Elimina luego la columna 'base'
    """
    to_drop = []
    # Iteramos sobre una copia de los nombres
    for col in list(df.columns):
        if "_" not in col:
            continue
        base, suffix = col.rsplit("_", 1)
        # si existe la columna base, unimos y marcamos para borrar la base
        if base in df.columns:
            df[col] = df[col].fillna(df[base])
            to_drop.append(base)
    return df.drop(columns=to_drop, errors="ignore")

In [22]:
df_check = postprocess_df(df_all)

/var/folders/f0/dt7lkxv100q68c369j1bzgk00000gn/T/ipykernel_18165/3967653526.py:30: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(_clean_cell)


In [23]:
import re

df_try = unify_suffix_columns(df_check)


In [24]:
df_try.shape

(8, 189)

In [25]:
list_missing = list(set(df_check.columns) - set(df_try.columns))
[i for i in list_missing if 'adj' in i]

['adjuntar_evidencia_de_cursos_de_nivel_de_ingles',
 'adjuntar_la_documentacion_de_los_visados_anteriores',
 'adjuntar_copia_de_tu_documento_nacional_de_identidad_o_cedula_de_identidad',
 'adjuntar_family_ties',
 'adjuntar_tu_seguro_actual']

In [42]:
list_missing[10:20]

['pais',
 'telefono_de_contacto_del_empleador_con_prefijo',
 'numero_telefono_en_tu_pais_de_residencia_habitual',
 'estado_o_provincia_en_donde_naciste',
 'has_estado_previamente_en_australia',
 'institucion_escuela_universidad',
 'tipo_de_industria',
 'adjuntar_copia_de_tu_documento_nacional_de_identidad_o_cedula_de_identidad',
 'fecha_de_expedicion',
 'entrada']

In [43]:
[i for i in df_all.columns if 'tipo_de_industria' in i]

['tipo_de_industria_te', 'tipo_de_industria']

In [44]:
f = 'tipo_de_industria'
df_check[[i for i in df_check.columns if f in i]]

,tipo_de_industria_te,tipo_de_industria
0,GESTIÓN DE TRÁNSITO,CONSTRUCCION
1,LABORATORIO FARMACEUTICO,FARMACEUTICA
2,Retail,No se ha subido ninguna respuesta.
3,Minera,None
4,Cleaning,cleaning construction
5,FERRETERA,FERRETERO; 21
6,Aged Care,Salud
7,HOSPITALITY,HOSPITALITY


In [40]:
df_try[[i for i in df_try.columns if f in i]]

,codigo_postal_ddc
0,05000
1,092302
2,48902
3,34224
4,730000
5,04009
6,7750000
7,520001


In [30]:
adjuntar_cols = [i for i in df_try.columns if 'adjuntar' in i]
[i for i in adjuntar_cols if 'ingles' in i]

['adjuntar_evidencia_de_cursos_de_nivel_de_ingles_ds']

In [37]:
df_all['salida'].loc[2]

'Indica cada fecha en el orden listado. Si no recuerdas el día, selecciona el mes y año correcto con; el día "1" e indícalo con un comentario desde el botón "Dudas?"; Senegal; Portugal; Italia; Francia; Marruecos; 28/6/2017; 1/8/2019; 14/10/2023; 3/12/2023; 29/1/2024'